# Working with robot


## I. Upload the robot model
In order to upload the model, one can utilise ready to use functions


In [ ]:
import os
import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
from os.path import dirname, join
import numpy as np

def getModelPath(subpath, printmsg=False):
    source = os.path.abspath('')  # top level directory
    if (printmsg): print("using %s as modelPath" % source)
    return source

class RobotLoader(object):
    path = ''
    urdf_filename = ''
    srdf_filename = ''
    urdf_subpath = 'urdf'
    srdf_subpath = 'srdf'
    ref_posture = 'half_sitting'
    has_rotor_parameters = False
    free_flyer = False
    verbose = False

    def __init__(self):
        urdf_path = join(self.path, self.urdf_subpath, self.urdf_filename)
        self.model_path = getModelPath(urdf_path, self.verbose)
        self.urdf_path = join(self.model_path, urdf_path)
        self.robot = RobotWrapper.BuildFromURDF(self.urdf_path, [join(self.model_path, '../..')],
                                                pin.JointModelFreeFlyer() if self.free_flyer else None)

        if self.srdf_filename:
            self.srdf_path = join(self.model_path, self.path, self.srdf_subpath, self.srdf_filename)
            self.robot.q0 = readParamsFromSrdf(self.robot.model, self.srdf_path, self.verbose,
                                               self.has_rotor_parameters, self.ref_posture)

            if pin.WITH_HPP_FCL and pin.WITH_HPP_FCL_BINDINGS:
                # Add all collision pairs
                self.robot.collision_model.addAllCollisionPairs()

                # Remove collision pairs per SRDF
                pin.removeCollisionPairs(self.robot.model, self.robot.collision_model, self.srdf_path, False)

                # Recreate collision data since the collision pairs changed
                self.robot.collision_data = self.robot.collision_model.createData()
        else:
            self.srdf_path = None
            self.robot.q0 = pin.neutral(self.robot.model)

        if self.free_flyer:
            self.addFreeFlyerJointLimits()

    def addFreeFlyerJointLimits(self):
        ub = self.robot.model.upperPositionLimit
        ub[:7] = 1
        self.robot.model.upperPositionLimit = ub
        lb = self.robot.model.lowerPositionLimit
        lb[:7] = -1
        self.robot.model.lowerPositionLimit = lb

    @property
    def q0(self):
        warnings.warn("`q0` is deprecated. Please use `robot.q0`", FutureWarning, 2)
        return self.robot.q0

def readParamsFromSrdf(model, SRDF_PATH, verbose=False, has_rotor_parameters=True, referencePose='half_sitting'):
    if has_rotor_parameters:
        pin.loadRotorParameters(model, SRDF_PATH, verbose)
    model.armature = np.multiply(model.rotorInertia.flat, np.square(model.rotorGearRatio.flat))
    pin.loadReferenceConfigurations(model, SRDF_PATH, verbose)
    q0 = pin.neutral(model)
    if referencePose is not None:
        q0 = model.referenceConfigurations[referencePose].copy()
    return q0


class FOREloader(RobotLoader):
    path = 'robot_description_package'
    urdf_filename = "robot_simplified.urdf"
    srdf_filename = "robot_simplified.srdf"
    ref_posture = "standing"
    free_flyer = True


ROBOTS = {
    'fore': FOREloader,
}

def load(name):
    """Load a robot by its name"""
    inst = ROBOTS[name]()
    return inst.robot

In [70]:
# Display the robot
fore = load('fore')
rmodel = fore.model
fore.initViewer(loadModel=True)
fore.display(fore.q0)

In [67]:
#All frames
for i,f in enumerate(rmodel.frames): print(i,f.name,f.parent)

0 universe 0
1 root_joint 1
2 base 1
3 base_to_base_inertial 2
4 base_inertial 2
5 LF_JHIP 3
6 LF_THIGH 3
7 LF_JKNEE 4
8 LF_CALF 4
9 LF_JANCKLE 5
10 LF_FOOT 5
11 LF_JFOOTPOINT 5
12 LF_FOOTPOINT 5


In [69]:
# Checking the joints
print(rmodel)
for i in rmodel.names:
    print(i)

Nb joints = 6 (nq=11,nv=10)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 base_to_base_inertial: parent=1
  Joint 3 LF_JHIP: parent=2
  Joint 4 LF_JKNEE: parent=3
  Joint 5 LF_JANCKLE: parent=4

universe
root_joint
base_to_base_inertial
LF_JHIP
LF_JKNEE
LF_JANCKLE


In [71]:
from pinocchio.utils import rand
rdata = rmodel.createData()
q = rand(rmodel.nq)
pin.forwardKinematics(rmodel,rdata,q)
for i,M in enumerate(rdata.oMi[1:]): print(i,M)
print("End effector = " , rdata.oMi[-1].translation.T)

0   R =
 -0.407921   0.227957    1.15618
    1.1784  -0.146065   0.204755
-0.0103421    1.17413  -0.430524
  p = 0.644688 0.290832  0.98608

1   R =
 -0.407921   0.227957    1.15618
    1.1784  -0.146065   0.204755
-0.0103421    1.17413  -0.430524
  p =  1.58612 0.457555 0.635523

2   R =
-0.677643  0.227957   1.02174
  1.09285 -0.146065  0.486035
 0.094995   1.17413 -0.420041
  p =  1.53369 0.665384 0.762712

3   R =
 -1.22011  0.227957  0.120369
 0.313808 -0.146065   1.15415
  0.38559   1.17413 -0.191778
  p =  1.39304 0.595805 0.833846

4   R =
  -1.17692   0.227957  -0.343538
 -0.139456  -0.146065     1.1879
  0.429299    1.17413 -0.0340693
  p =    1.375 0.447299 0.846449



In [78]:
# random configuration display
# problem with randomised quaternion values
from pinocchio.utils import rand
fore.display(rand(fore.model.nq)*2-1)
fore.model.nq

End effector =  [1.37499742 0.44729889 0.84644895]


11

### Uploading and displaying the ANYmal robot 


In [41]:
import numpy as np
import crocoddyl
import example_robot_data
robot = example_robot_data.load('anymal')
robot_model = robot.model
target = np.array([0.4, 0., .4])
display = crocoddyl.GepettoDisplay(robot)
display.robot.viewer.gui.applyConfiguration('world/point', target.tolist() + [0., 0., 0., 1.])  # xyz+quaternion
display.robot.viewer.gui.refresh()


In [42]:
from pinocchio.utils import rand,zero,eye
# robot.display(rand(robot.model.nq)*2-1)
robot.display(robot.q0)


## II. Jumping 


In [73]:
import crocoddyl
import pinocchio
import numpy as np

class SimpleQuadrupedalGaitProblem:
    
    def __init__(self, rmodel, feet): # for ANYmal: feet = [lfFoot, rfFoot, lhFoot, rhFoot]
        self.rmodel = rmodel
        self.rdata = rmodel.createData()
        self.state = crocoddyl.StateMultibody(self.rmodel)
        self.actuation = crocoddyl.ActuationModelFloatingBase(self.state)
        # Getting the frame id for all the legs
        self.feet = []
        for foot in feet:
            self.feet +=[self.rmodel.getFrameId(foot)]
#         self.lfFootId = self.feet[0]
#         self.rfFootId = self.feet[1]
#         self.lhFootId = self.feet[2]
#         self.rhFootId = self.feet[3]
        # Defining default state
        q0 = self.rmodel.referenceConfigurations["standing"]
        self.rmodel.defaultState = np.concatenate([q0, np.zeros(self.rmodel.nv)])
        self.firstStep = True
        # Defining the friction coefficient and normal
        self.mu = 0.7
        self.Rsurf = np.eye(3)
        
    def createJumpingProblem(self, x0, jumpHeight, jumpLength, timeStep, groundKnots, flyingKnots):
        q0 = x0[:self.rmodel.nq]
        pinocchio.forwardKinematics(self.rmodel, self.rdata, q0)
        pinocchio.updateFramePlacements(self.rmodel, self.rdata)
        FootPos0 = []
        comRef = 0
        for foot in self.feet:
            FootPos0 += [self.rdata.oMf[foot].translation]
            FootPos0[-1][2] = 0
#             comRef += FootPos0[-1]
#         comRef /= 4
        comRef = pinocchio.centerOfMass(self.rmodel, self.rdata, q0) # assumption is that the intitial com is correct
        
#         rfFootPos0 = self.rdata.oMf[self.feet[1]].translation
#         rhFootPos0 = self.rdata.oMf[self.feet[3]].translation
#         lfFootPos0 = self.rdata.oMf[self.feet[0]].translation
#         lhFootPos0 = self.rdata.oMf[self.feet[2]].translation
# #         df = jumpLength[2] - rfFootPos0[2]
#         rfFootPos0[2] = 0.
#         rhFootPos0[2] = 0.
#         lfFootPos0[2] = 0.
#         lhFootPos0[2] = 0.
        
#         comRef = (rfFootPos0 + rhFootPos0 + lfFootPos0 + lhFootPos0) / 4
        

        loco3dModel = []
        takeOff = [
            self.createSwingFootModel(
                timeStep,
                self.feet,
            ) for k in range(groundKnots)
        ]
        flyingUpPhase = [
            self.createSwingFootModel(
                timeStep, [],
                np.array([0., 0., jumpHeight]) * (k + 1) / flyingKnots + comRef)
            for k in range(flyingKnots)
        ]
        flyingDownPhase = []
        for k in range(flyingKnots):
            flyingDownPhase += [self.createSwingFootModel(timeStep, [])]

#          f0 = jumpLength
#         footTask = [
#             crocoddyl.FramePlacement(self.lfFootId, pinocchio.SE3(np.eye(3), lfFootPos0 + f0)),
#             crocoddyl.FramePlacement(self.rfFootId, pinocchio.SE3(np.eye(3), rfFootPos0 + f0)),
#             crocoddyl.FramePlacement(self.lhFootId, pinocchio.SE3(np.eye(3), lhFootPos0 + f0)),
#             crocoddyl.FramePlacement(self.rhFootId, pinocchio.SE3(np.eye(3), rhFootPos0 + f0))
#         ]
#         landingPhase = [
#             self.createFootSwitchModel([self.lfFootId, self.rfFootId, self.lhFootId, self.rhFootId], footTask, False)
#         ]
#         f0[2] = df
        landed = [
            self.createSwingFootModel(timeStep, self.feet,
                                      comTask=comRef) for k in range(groundKnots)
        ]
        loco3dModel += takeOff
        loco3dModel += flyingUpPhase
        loco3dModel += flyingDownPhase
#         loco3dModel += landingPhase
        loco3dModel += landed

        problem = crocoddyl.ShootingProblem(x0, loco3dModel, loco3dModel[-1])
        return problem
    
    def createSwingFootModel(self, timeStep, supportFootIds, comTask=None, swingFootTask=None):
        """ Action model for a swing foot phase.

        :param timeStep: step duration of the action model
        :param supportFootIds: Ids of the constrained feet
        :param comTask: CoM task
        :param swingFootTask: swinging foot task
        :return action model for a swing foot phase
        """
        # Creating a 3D multi-contact model, and then including the supporting
        # foot
        contactModel = crocoddyl.ContactModelMultiple(self.state, self.actuation.nu)
        for i in supportFootIds:
            xref = crocoddyl.FrameTranslation(i, np.array([0., 0., 0.]))
            supportContactModel = crocoddyl.ContactModel3D(self.state, xref, self.actuation.nu, np.array([0., 50.]))
            contactModel.addContact(self.rmodel.frames[i].name + "_contact", supportContactModel)

        # Creating the cost model for a contact phase
        costModel = crocoddyl.CostModelSum(self.state, self.actuation.nu)
        if isinstance(comTask, np.ndarray):
            comResidual = crocoddyl.ResidualModelCoMPosition(self.state, comTask, self.actuation.nu)
            comTrack = crocoddyl.CostModelResidual(self.state, comResidual)
            costModel.addCost("comTrack", comTrack, 1e6)
        for i in supportFootIds:
            cone = crocoddyl.FrictionCone(self.Rsurf, self.mu, 4, False)
            coneResidual = crocoddyl.ResidualModelContactFrictionCone(self.state, i, cone, self.actuation.nu)
            coneActivation = crocoddyl.ActivationModelQuadraticBarrier(crocoddyl.ActivationBounds(cone.lb, cone.ub))
            frictionCone = crocoddyl.CostModelResidual(self.state, coneActivation, coneResidual)
            costModel.addCost(self.rmodel.frames[i].name + "_frictionCone", frictionCone, 1e1)
        if swingFootTask is not None:
            for i in swingFootTask:
                frameTranslationResidual = crocoddyl.ResidualModelFrameTranslation(self.state, i.id,
                                                                                   i.placement.translation,
                                                                                   self.actuation.nu)
                footTrack = crocoddyl.CostModelResidual(self.state, frameTranslationResidual)
                costModel.addCost(self.rmodel.frames[i.id].name + "_footTrack", footTrack, 1e6)

        stateWeights = np.array([0.] * 3 + [500.] * 3 + [0.01] * (self.rmodel.nv - 6) + [10.] * 6 + [1.] *
                                (self.rmodel.nv - 6))
        stateResidual = crocoddyl.ResidualModelState(self.state, self.rmodel.defaultState, self.actuation.nu)
        stateActivation = crocoddyl.ActivationModelWeightedQuad(stateWeights**2)
        ctrlResidual = crocoddyl.ResidualModelControl(self.state, self.actuation.nu)
        stateReg = crocoddyl.CostModelResidual(self.state, stateActivation, stateResidual)
        ctrlReg = crocoddyl.CostModelResidual(self.state, ctrlResidual)
        costModel.addCost("stateReg", stateReg, 1e1)
        costModel.addCost("ctrlReg", ctrlReg, 1e-1)

        lb = np.concatenate([self.state.lb[1:self.state.nv + 1], self.state.lb[-self.state.nv:]])
        ub = np.concatenate([self.state.ub[1:self.state.nv + 1], self.state.ub[-self.state.nv:]])
        stateBoundsResidual = crocoddyl.ResidualModelState(self.state, 0 * self.rmodel.defaultState, self.actuation.nu)
        stateBoundsActivation = crocoddyl.ActivationModelQuadraticBarrier(crocoddyl.ActivationBounds(lb, ub))
        stateBounds = crocoddyl.CostModelResidual(self.state, stateBoundsActivation, stateBoundsResidual)
        costModel.addCost("stateBounds", stateBounds, 1e3)

        # Creating the action model for the KKT dynamics with simpletic Euler
        # integration scheme
        dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(self.state, self.actuation, contactModel,
                                                                     costModel, 0., True)
        model = crocoddyl.IntegratedActionModelEuler(dmodel, timeStep)
        return model
    

In [74]:
def plotSolutionOneLeg(solver, bounds = True):
    rmodel = solver.problem.runningModels[0].state.pinocchio
    xs, us = solver.xs, solver.us
    import matplotlib.pyplot as plt
    if bounds:
        us_lb, us_ub = [], []
        xs_lb, xs_ub = [], []
    models = solver.problem.runningModels.tolist() + [solver.problem.terminalModel]
    for m in models:
        us_lb += [m.u_lb]
        us_ub += [m.u_ub]
        xs_lb += [m.state.lb]
        xs_ub += [m.state.ub]

    # Getting the state and control trajectories
    nx, nq, nu = xs[0].shape[0], rmodel.nq, us[0].shape[0]
    X = [0.] * nx
    U = [0.] * nu
    if bounds:
        U_LB = [0.] * nu
        U_UB = [0.] * nu
        X_LB = [0.] * nx
        X_UB = [0.] * nx
    for i in range(nx):
        X[i] = [np.asscalar(x[i]) for x in xs]
        if bounds:
            X_LB[i] = [np.asscalar(x[i]) for x in xs_lb]
            X_UB[i] = [np.asscalar(x[i]) for x in xs_ub]
    for i in range(nu):
        U[i] = [np.asscalar(u[i]) if u.shape[0] != 0 else 0 for u in us]
        if bounds:
            U_LB[i] = [np.asscalar(u[i]) if u.shape[0] != 0 else np.nan for u in us_lb]
            U_UB[i] = [np.asscalar(u[i]) if u.shape[0] != 0 else np.nan for u in us_ub]

    # Plotting the joint positions, velocities and torques
    plt.figure()
    legJointNames = ['HAA', 'HFE', 'KFE']
    # LF foot
    plt.subplot(1, 3, 1)
    plt.title('joint position [rad]')
    [plt.plot(X[k], label=legJointNames[i]) for i, k in enumerate(range(7, 10))]
    if bounds:
        [plt.plot(X_LB[k], '--r') for i, k in enumerate(range(7, 10))]
        [plt.plot(X_UB[k], '--r') for i, k in enumerate(range(7, 10))]
    plt.ylabel('LF')
    plt.legend()
    plt.subplot(1, 3, 2)
    plt.title('joint velocity [rad/s]')
    [plt.plot(X[k], label=legJointNames[i]) for i, k in enumerate(range(nq + 6, nq + 9))]
    if bounds:
        [plt.plot(X_LB[k], '--r') for i, k in enumerate(range(nq + 6, nq + 9))]
        [plt.plot(X_UB[k], '--r') for i, k in enumerate(range(nq + 6, nq + 9))]
    plt.ylabel('LF')
    plt.legend()
    plt.subplot(1, 3, 3)
    plt.title('joint torque [Nm]')
    [plt.plot(U[k], label=legJointNames[i]) for i, k in enumerate(range(0, 3))]
    if bounds:
        [plt.plot(U_LB[k], '--r') for i, k in enumerate(range(0, 3))]
        [plt.plot(U_UB[k], '--r') for i, k in enumerate(range(0, 3))]
    plt.ylabel('LF')
    plt.legend()
    
    us_lb, us_ub = [], []
    xs_lb, xs_ub = [], []        
    
    plt.figure()
    plt.suptitle('com')
    rdata = rmodel.createData()
    Cx = []
    Cy = []
    for x in xs:
        q = x[:rmodel.nq]
        c = pinocchio.centerOfMass(rmodel, rdata, q)
        Cx.append(np.asscalar(c[0]))
        Cy.append(np.asscalar(c[1]))
    plt.plot(Cx, Cy)
    plt.title('CoM position')
    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.grid(True)
    plt.show()

### Using anymal for jumping


In [45]:
%matplotlib qt
from crocoddyl.utils.quadruped import plotSolution
DISPLAY = True
PLOT = True
import example_robot_data
# Loading the anymal model
anymal = example_robot_data.load('anymal')

# Defining the initial state of the robot
q0 = anymal.model.referenceConfigurations['standing'].copy()
v0 = pinocchio.utils.zero(anymal.model.nv)
x0 = np.concatenate([q0, v0])

# Setting up the 3d walking problem
lfFoot, rfFoot, lhFoot, rhFoot = 'LF_FOOT', 'RF_FOOT', 'LH_FOOT', 'RH_FOOT'
gait = SimpleQuadrupedalGaitProblem(anymal.model, [lfFoot, rfFoot, lhFoot, rhFoot])

# Setting up all tasks
GAITPHASES = {
    'jumping': {
        'jumpHeight': 0.15,
        'jumpLength': [0.0, 0.3, 0.],
        'timeStep': 1e-2,
        'groundKnots': 10,
        'flyingKnots': 20
    }
}

cameraTF = [2., 2.68, 0.84, 0.2, 0.62, 0.72, 0.22]

value = GAITPHASES['jumping']
ddp = crocoddyl.SolverFDDP(
    gait.createJumpingProblem(x0, value['jumpHeight'], value['jumpLength'], value['timeStep'],
                              value['groundKnots'], value['flyingKnots']))


# ddp.setCallbacks(
#     [crocoddyl.CallbackLogger(),
# #      crocoddyl.CallbackVerbose(),
#      crocoddyl.CallbackDisplay(display)])

    
if DISPLAY:
    # Show the solution itterations
    display = crocoddyl.GepettoDisplay(anymal, 4, 4, cameraTF, frameNames=[lfFoot, rfFoot, lhFoot, rhFoot])

if PLOT or DISPLAY:
    ddp.setCallbacks(
        [crocoddyl.CallbackLogger(),
         crocoddyl.CallbackVerbose(),
         crocoddyl.CallbackDisplay(display)])
    
    
xs = [anymal.model.defaultState] * (ddp.problem.T + 1)
us = ddp.problem.quasiStatic([anymal.model.defaultState] * ddp.problem.T)
solution = ddp.solve(xs, us, 100, False, 0.1)
print(solution)

if DISPLAY:
    # Show the final result
    display = crocoddyl.GepettoDisplay(anymal, frameNames=[lfFoot, rfFoot, lhFoot, rhFoot])
    display.displayFromSolver(ddp)



# Plotting the entire motion
if PLOT:
    log = ddp.getCallbacks()[0]
    plotSolution(ddp, figIndex=1, show=False)
    
    title = list(GAITPHASES.keys())[0] + " (phase " + str(0) + ")"
    crocoddyl.plotConvergence(log.costs,
                              log.u_regs,
                              log.x_regs,
                              log.grads,
                              log.stops,
                              log.steps,
                              figTitle=title,
                              figIndex=3,
                              show=True)


True


### Using FORE robot configuration


In [77]:
%matplotlib qt
DISPLAY = True
PLOT = True
fore = load('fore')
from crocoddyl.utils.quadruped import plotSolution

# Defining the initial state of the robot
q0 = fore.model.referenceConfigurations['standing'].copy()
v0 = pinocchio.utils.zero(fore.model.nv)
x0 = np.concatenate([q0, v0])
lfFoot = 'LF_FOOTPOINT'
gait = SimpleQuadrupedalGaitProblem(fore.model,[lfFoot])

# Setting up all tasks
GAITPHASES = {
    'jumping': {
        'jumpHeight': 0.15,
        'jumpLength': [0.0, 0.3, 0.],
        'timeStep': 1e-2,
        'groundKnots': 10,
        'flyingKnots': 20
    }
}

cameraTF = [2., 2.68, 0.84, 0.2, 0.62, 0.72, 0.22]

value = GAITPHASES['jumping']
ddp = crocoddyl.SolverFDDP(
    gait.createJumpingProblem(x0, value['jumpHeight'], value['jumpLength'], value['timeStep'],
                              value['groundKnots'], value['flyingKnots']))

# Show the solution itterations
display = crocoddyl.GepettoDisplay(fore, 4, 4, cameraTF, frameNames=[lfFoot])
ddp.setCallbacks([crocoddyl.CallbackVerbose(), crocoddyl.CallbackDisplay(display)])

xs = [fore.model.defaultState] * (ddp.problem.T + 1)
us = ddp.problem.quasiStatic([fore.model.defaultState] * ddp.problem.T)
solution = ddp.solve(xs, us, 100, False, 0.1)
print(solution)
# Show the final result
display = crocoddyl.GepettoDisplay(fore, frameNames=[lfFoot])
display.displayFromSolver(ddp)

if DISPLAY:
    # Show the solution itterations
    display = crocoddyl.GepettoDisplay(fore, 4, 4, cameraTF, frameNames=[lfFoot])

if PLOT or DISPLAY:
    ddp.setCallbacks(
        [crocoddyl.CallbackLogger(),
         crocoddyl.CallbackVerbose(),
         crocoddyl.CallbackDisplay(display)])
    
    
xs = [fore.model.defaultState] * (ddp.problem.T + 1)
us = ddp.problem.quasiStatic([fore.model.defaultState] * ddp.problem.T)
solution = ddp.solve(xs, us, 100, False, 0.1)
print(solution)

if DISPLAY:
    # Show the final result
    display = crocoddyl.GepettoDisplay(fore, frameNames=[lfFoot])
    display.displayFromSolver(ddp)

False
False


In [47]:
# Plotting the entire motion for fore
if PLOT:
    log = ddp.getCallbacks()[0]
#     plotSolution(ddp, figIndex=1, show=False) # error expected since the function expects 4 legs
    plotSolutionOneLeg(ddp)
    title = list(GAITPHASES.keys())[0] + " (phase " + str(0) + ")"
    crocoddyl.plotConvergence(log.costs, 
                              log.u_regs,
                              log.x_regs,
                              log.grads,
                              log.stops,
                              log.steps,
                              figTitle=title,
                              figIndex=3,
                              show=True)

<ipython-input-44-ba1294c2b0bc>:25: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  X[i] = [np.asscalar(x[i]) for x in xs]
<ipython-input-44-ba1294c2b0bc>:27: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  X_LB[i] = [np.asscalar(x[i]) for x in xs_lb]
<ipython-input-44-ba1294c2b0bc>:28: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  X_UB[i] = [np.asscalar(x[i]) for x in xs_ub]
<ipython-input-44-ba1294c2b0bc>:30: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  U[i] = [np.asscalar(u[i]) if u.shape[0] != 0 else 0 for u in us]
<ipython-input-44-ba1294c2b0bc>:32: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  U_LB[i] = [np.asscalar(u[i]) if u.shape[0] != 0 else np.nan for u in us_lb]
<ipython-input-44-ba1294c2b0bc>:33: DeprecationWarning: np.asscalar(a) is deprecated since NumP